In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
from math import *
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import warnings; warnings.simplefilter('ignore')
init_notebook_mode(connected=True)

In [19]:
def read_data(date,index="SPY",indicator="impliedV"):
    path = os.path.join("data",indicator,index,date+"_"+index+"~market__"+indicator+".csv")
    df = pd.read_csv(path)
    return(df)

# volatility surface :

In [20]:
def plot_vol_surface(df,x_col="expiry",y_col="strike",target_z_col="midImpliedV"):
    df = df.reset_index()
    df["expiry"] = df["expiry"].astype("str")
    df["expiry"] = pd.to_datetime(df["expiry"],format="%Y-%m-%d")
    df_pivot = df.pivot(index=x_col,columns=y_col,values=target_z_col)
    fig = go.Figure(data=[go.Surface(x=df_pivot.columns.values,y=df_pivot.index ,z=df_pivot.values, connectgaps=True)])

    fig.update_layout(title='volatility surface', autosize=True,
                      width=800, height=600)

    fig.show()

In [4]:
df = read_data(index="SPY",date="20130814")
plot_vol_surface(df)

# calibration of implied volatility model ( quadratic polynomial regression)

In [21]:
def get_iv_atm(expiry,df):
    df1 = df[df.expiry == expiry]
    df_bis = df1.drop_duplicates("expiry")
    df_bis["strike"] = df_bis["fwd"]
    df_bis["midImpliedV"] = np.nan
    df1 = df1.append(df_bis,ignore_index=True)
    df1.sort_values("strike",inplace=True)
    df1["midImpliedV"] = df1["midImpliedV"].interpolate()
    return(df1[df1["strike"] == df1["fwd"]]["midImpliedV"].values[0])

In [22]:
def add_iv_atm_col(df):
    def change_iv_atm(r,expiry,v):
        if r["expiry"]==expiry:
            r["IV_ATM"]=v
        return r
    # compute of IV_ATM of each expiry and add it to the column df["IV_ATM"]
    df["IV_ATM"] = np.nan
    for expiry in df.expiry.unique():
        v = get_iv_atm(expiry,df)
        df = df.apply(lambda r: change_iv_atm(r,expiry,v), axis=1)
    return df


In [23]:
def Calibration_fixed_date(date,index="SPY",indicator="impliedV"):
    def change_level_skew_conv(r,expiry,lev,sk,conv,i):
        if r["expiry"]==expiry:
            r["Level_"+str(i)] = lev
            r["Skew_"+str(i)] = sk
            r["Convex_"+str(i)] = conv
        return r
    df = read_data(date=date,index=index,indicator=indicator)
    df = add_iv_atm_col(df)
    df["Moneyness_2"] = np.log(df["strike"]/df["fwd"])/(np.sqrt(df["tenor"])*df["IV_ATM"])
    for i in [2]:
        df["Level_"+str(i)] = np.nan
        df["Skew_"+str(i)] = np.nan
        df["Convex_"+str(i)] = np.nan
        for expiry in df.expiry.unique():
            df1 = df[df["expiry"] == expiry]
            params = np.polyfit(df1["Moneyness_"+str(i)], df1["midImpliedV"], 2)
            conv,skew,lev = params
            df = df.apply(lambda r: change_level_skew_conv(r,expiry,lev,skew,conv,i), axis=1)
    df = df.drop_duplicates("expiry")
    df = df.reset_index()
    df["date"] = date
    df = df[["date","expiry","dow","tenor","vTenor","spot","fwd","IV_ATM","Level_2","Skew_2","Convex_2"]]
    return df

In [24]:
def Calibration_all_dates(index="SPY",indicator="impliedV",start_year=None,end_year="9999",nb_days=None):
    df_all = pd.DataFrame()
    l = os.listdir(os.path.join("data",indicator,index))
    l.sort()
    if start_year is not None:
        regex = re.compile(r'^[%a-%a][%a-%a][%a-%a][%a-%a][0-9]{2}[0-9]{2}'%(int(start_year[0]),int(end_year[0]),
                                                                             int(start_year[1]),int(end_year[1]),
                                                                             int(start_year[2]),int(end_year[2]),
                                                                             int(start_year[3]),int(end_year[3])))
        l = list(filter(regex.search, l))
    if nb_days is not None:
        l = l[:nb_days]
    for file in l:
        try:
            date = file[:8]
            df = Calibration_fixed_date(date,index=index,indicator=indicator)
            df_all = df_all.append(df, ignore_index=True)
        except:
            pass
    df_all.sort_values(["date","expiry"],inplace=True)
    return(df_all)

In [25]:
df_all = Calibration_all_dates(index="IWM",indicator="impliedV",nb_days=None, start_year="2013", end_year="2013")

In [26]:
df_all.to_csv("SPY_2013_ImpliedVStatistics.csv",index=False)

In [27]:
df_all = pd.read_csv("SPY_2013_ImpliedVStatistics.csv")
df_all["date"] = df_all["date"].astype(str)

In [32]:
df_all.head()

,date,expiry,dow,tenor,vTenor,spot,fwd,IV_ATM,Level_2,Skew_2,Convex_2
0,20130102,20130104,F,0.005479,0.005845,86.744371,86.788559,0.161822,0.160477,-0.018455,0.000182
1,20130102,20130111,F,0.024658,0.025135,86.744371,86.769886,0.154344,0.155148,-0.025682,0.002502
2,20130102,20130119,F,0.043836,0.044425,86.744371,86.766336,0.160105,0.167884,-0.025502,0.002279
3,20130102,20130125,F,0.063014,0.063130,86.744371,86.752753,0.160122,0.164113,-0.026890,0.002107
4,20130102,20130201,F,0.082192,0.082420,86.744371,86.749509,0.167865,0.168393,-0.031396,0.001581


In [37]:
df_all["date"].unique()

array(['20130102', '20130103', '20130104', '20130107', '20130108',
       '20130109', '20130110', '20130111', '20130114', '20130115',
       '20130116', '20130117', '20130118', '20130122', '20130123',
       '20130124', '20130125', '20130128', '20130129', '20130130',
       '20130131', '20130201', '20130204', '20130205', '20130206',
       '20130207', '20130208', '20130211', '20130212', '20130213',
       '20130214', '20130215', '20130219', '20130220', '20130221',
       '20130222', '20130225', '20130226', '20130227', '20130228',
       '20130301', '20130304', '20130305', '20130306', '20130307',
       '20130308', '20130311', '20130312', '20130313', '20130314',
       '20130315', '20130318', '20130319', '20130320', '20130321',
       '20130322', '20130325', '20130326', '20130327', '20130328',
       '20130401', '20130402', '20130403', '20130404', '20130405',
       '20130408', '20130409', '20130410', '20130411', '20130412',
       '20130415', '20130416', '20130417', '20130418', '201304

In [29]:
def plot_params_vs_tenor(df_all, date):
    
    df_date = df_all[df_all["date"]==date]
    fig = make_subplots(rows=1, cols=3, subplot_titles=["Level vs tenor","Skew vs tenor","Convex vs tenor"])
    
    for i,param in enumerate(["Level_2","Skew_2","Convex_2"]): 
        
        fig.add_trace(go.Scatter(x=df_date["tenor"], y=df_date[param], mode='markers+lines', name=param)
                                 ,row=1,col=i+1)
    
    fig.update_layout(height = 350,
                title_text="date : "+date)
    fig.show()
    


In [38]:
plot_params_vs_tenor(df_all,"20130107")

In [39]:
plot_params_vs_tenor(df_all,"20130313")

In [40]:
plot_params_vs_tenor(df_all,"20130829")

In [95]:
plot_params_vs_tenor("20100318")

In [43]:
plot_params_vs_tenor(df_all,"20130909")

In [42]:
plot_params_vs_tenor(df_all,"20131216")

In [44]:
max_maturity_days = 350
df_all_sort = df_all.sort_values("tenor")
df_to_plot = df_all_sort.groupby("tenor").mean()
fig = make_subplots(rows=3,cols=1,subplot_titles=["Level vs tenor","Skew vs tenor","Convex vs tenor"])

for i,param in enumerate(["Level_2","Skew_2","Convex_2"]):
    fig.add_trace(go.Scatter(
        x=df_to_plot.index.values, y=df_to_plot[param].values,mode = "markers+lines",
                            fillcolor = "blue",
                            opacity = 0.4),i+1,1)
    fig.update_yaxes(title_text=param, row=1+i, col=1)
    fig.update_xaxes(title_text="tenor", row=1+i, col=1)
    
fig.update_layout( height = 1100, title = "Mean over all the dates of the parameters for each fixed tenor",
                  showlegend = False)
iplot(fig)